In [41]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [42]:
import numpy as np
import pandas as pd

In [43]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [44]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [45]:
# reflect an existing database into a new model
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [46]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [47]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [48]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [49]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
#Starting from the last data point in the database. 
prev_year = dt.date(2017, 8, 23)

# Calculate the date one year from the last date in data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.station, Measurement.prcp, Measurement.tobs).filter(Measurement.date >= prev_year).all()
print(results)

# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


[('2016-08-23', 'USC00519397', 0.0, 81.0), ('2016-08-24', 'USC00519397', 0.08, 79.0), ('2016-08-25', 'USC00519397', 0.08, 80.0), ('2016-08-26', 'USC00519397', 0.0, 79.0), ('2016-08-27', 'USC00519397', 0.0, 77.0), ('2016-08-28', 'USC00519397', 0.01, 78.0), ('2016-08-29', 'USC00519397', 0.0, 78.0), ('2016-08-30', 'USC00519397', 0.0, 79.0), ('2016-08-31', 'USC00519397', 0.13, 80.0), ('2016-09-01', 'USC00519397', 0.0, 81.0), ('2016-09-02', 'USC00519397', 0.0, 80.0), ('2016-09-03', 'USC00519397', 0.0, 79.0), ('2016-09-04', 'USC00519397', 0.03, 75.0), ('2016-09-05', 'USC00519397', None, 79.0), ('2016-09-06', 'USC00519397', None, 76.0), ('2016-09-07', 'USC00519397', 0.05, 76.0), ('2016-09-08', 'USC00519397', 0.0, 80.0), ('2016-09-09', 'USC00519397', 0.03, 79.0), ('2016-09-10', 'USC00519397', 0.0, 78.0), ('2016-09-11', 'USC00519397', 0.05, 76.0), ('2016-09-12', 'USC00519397', 0.0, 78.0), ('2016-09-13', 'USC00519397', 0.02, 78.0), ('2016-09-14', 'USC00519397', 1.32, 75.0), ('2016-09-15', 'USC00

In [50]:
df = pd.DataFrame(results, columns=['date','station', 'precipitation', 'temp'])
df

,date,station,precipitation,temp
0,2016-08-23,USC00519397,0.00,81.0
1,2016-08-24,USC00519397,0.08,79.0
2,2016-08-25,USC00519397,0.08,80.0
3,2016-08-26,USC00519397,0.00,79.0
4,2016-08-27,USC00519397,0.00,77.0
...,...,...,...,...
2225,2017-08-19,USC00516128,0.09,71.0
2226,2017-08-20,USC00516128,NaN,78.0
2227,2017-08-21,USC00516128,0.56,76.0
2228,2017-08-22,USC00516128,0.50,76.0


In [51]:
df.describe()

,precipitation,temp
count,2021.000000,2230.000000
mean,0.177279,74.590583
std,0.461190,4.611667
min,0.000000,58.000000
25%,0.000000,72.000000
50%,0.020000,75.000000
75%,0.130000,78.000000
max,6.700000,87.000000


In [52]:
df.dtypes

date              object
station           object
precipitation    float64
temp             float64
dtype: object

In [53]:
df['date'] = df['date'].apply(pd.to_datetime)

In [54]:
df.dtypes

date             datetime64[ns]
station                  object
precipitation           float64
temp                    float64
dtype: object

In [57]:
pd.DatetimeIndex(df.date).to_period("M")

PeriodIndex(['2016-08', '2016-08', '2016-08', '2016-08', '2016-08', '2016-08',
             '2016-08', '2016-08', '2016-08', '2016-09',
             ...
             '2017-08', '2017-08', '2017-08', '2017-08', '2017-08', '2017-08',
             '2017-08', '2017-08', '2017-08', '2017-08'],
            dtype='period[M]', name='date', length=2230, freq='M')

In [60]:
df.groupby(pd.Grouper(key='date',freq='M')).size()

date
2016-08-31     59
2016-09-30    190
2016-10-31    198
2016-11-30    186
2016-12-31    200
2017-01-31    191
2017-02-28    177
2017-03-31    189
2017-04-30    190
2017-05-31    174
2017-06-30    191
2017-07-31    194
2017-08-31     91
Freq: M, dtype: int64

In [61]:
df.groupby(
    pd.Grouper(key='date',freq='M')
)['precipitation'].apply(sum)

date
2016-08-31    20.81
2016-09-30    49.40
2016-10-31    21.77
2016-11-30    20.56
2016-12-31    35.51
2017-01-31    11.95
2017-02-28    39.06
2017-03-31    29.62
2017-04-30    54.91
2017-05-31    25.27
2017-06-30    20.52
2017-07-31    23.72
2017-08-31     5.18
Freq: M, Name: precipitation, dtype: float64

In [71]:
df.groupby(
    pd.Grouper(key='date',freq='M')
)['temp'].apply(min)

date
2016-08-31    71.0
2016-09-30    70.0
2016-10-31    68.0
2016-11-30    63.0
2016-12-31    60.0
2017-01-31    60.0
2017-02-28    59.0
2017-03-31    62.0
2017-04-30    58.0
2017-05-31    65.0
2017-06-30    71.0
2017-07-31    69.0
2017-08-31    70.0
Freq: M, Name: temp, dtype: float64

In [72]:
df.groupby(pd.Grouper(key='date',freq='M'))['temp'].describe()

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2016-08-31,59.0,78.135593,2.555978,71.0,77.0,78.0,80.0,84.0
2016-09-30,190.0,77.747368,2.604304,70.0,76.0,78.0,79.0,84.0
2016-10-31,198.0,77.196970,2.587865,68.0,76.0,77.5,79.0,82.0
2016-11-30,186.0,74.543011,2.887209,63.0,73.0,75.0,77.0,80.0
2016-12-31,200.0,71.130000,3.419292,60.0,69.0,71.0,74.0,78.0
2017-01-31,191.0,69.691099,4.088993,60.0,67.0,70.0,73.0,81.0
2017-02-28,177.0,69.892655,4.643520,59.0,67.0,70.0,73.0,80.0
2017-03-31,189.0,72.370370,3.734595,62.0,70.0,73.0,75.0,82.0
2017-04-30,190.0,73.652632,4.070200,58.0,71.0,74.0,77.0,83.0


In [73]:
df.groupby(pd.Grouper(key='date',freq='M'))['precipitation'].describe()

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2016-08-31,53.0,0.392642,0.701378,0.0,0.01,0.08,0.2400,2.46
2016-09-30,173.0,0.285549,0.662214,0.0,0.01,0.07,0.2700,6.70
2016-10-31,179.0,0.121620,0.344809,0.0,0.00,0.01,0.0800,3.46
2016-11-30,171.0,0.120234,0.342695,0.0,0.00,0.02,0.0900,2.87
2016-12-31,178.0,0.199494,0.342967,0.0,0.02,0.06,0.2475,2.37
2017-01-31,170.0,0.070294,0.276047,0.0,0.00,0.00,0.0175,2.64
2017-02-28,162.0,0.241111,0.672922,0.0,0.00,0.00,0.0600,5.04
2017-03-31,169.0,0.175266,0.431165,0.0,0.00,0.00,0.0800,2.40
2017-04-30,171.0,0.321111,0.702469,0.0,0.00,0.02,0.3100,6.25


# Challenge